In [ ]:

# MUST4Water Interactive Hydrology-Economy Simulation

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider, IntSlider, Dropdown, Checkbox, FileUpload
import io

# SSP scenario parameters
SSP_PARAMS = {
    'SSP2 (baseline)': dict(gamma=1.428, P_shift=0, E_shift=0, M=0.7),
    'SSP3 (regional rivalry)': dict(gamma=1.3, P_shift=-50, E_shift=50, M=0.5),
    'SSP5 (fossil development)': dict(gamma=1.5, P_shift=20, E_shift=80, M=0.8),
}

def generate_synthetic_data(n_years, mean_P=949.88, sd_P=131.31, shift_P=0, shift_E=0):
    P = np.random.normal(mean_P + shift_P, sd_P, n_years)
    E = 0.1548 * P + 345.84 + shift_E + np.random.normal(0, 909.74, size=n_years)
    R = 3.161085 + 0.459239 * P - 0.4023 * E + np.random.normal(0, 100.231, size=n_years)
    I = 29.12463 + 0.485631 * P - 0.556753 * E + np.random.normal(0, 96.59519, size=n_years)
    return P, E, R, I

def feasible_runoff(Rt, R_bar, ER=0.2, MR=0.7):
    if ER * R_bar <= Rt <= (MR + ER) * R_bar:
        return Rt
    elif Rt > (MR + ER) * R_bar:
        return MR * R_bar
    else:
        return 0

def feasible_recharge(It, I_bar, B=0.13):
    if It < I_bar * (1 - B):
        return I_bar * (1 - B)
    elif It > I_bar * (1 + B):
        return I_bar * (1 + B)
    else:
        return It

def adjust_agriculture_coeff(f_gw, f_sw, eta_gw, eta_sw, f_green, P, E, P_bar, E_bar, gamma):
    if P < P_bar:
        f_green_new = ((P_bar - P) / P_bar) * f_green
        f_gw_new = f_gw + eta_gw * ((P_bar - P) / P_bar) * f_green * gamma + eta_gw * ((E - E_bar) / E_bar) * f_gw
        f_sw_new = f_sw + eta_sw * ((P_bar - P) / P_bar) * f_green * gamma + eta_sw * ((E - E_bar) / E_bar) * f_sw
    else:
        f_green_new = 0
        f_gw_new = f_gw + eta_gw * ((E - E_bar) / E_bar) * f_gw
        f_sw_new = f_sw + eta_sw * ((E - E_bar) / E_bar) * f_sw
    return f_gw_new, f_sw_new, f_green_new

def restitution_agriculture(f_gw_new, f_sw_new, f_gw, f_sw, r_gw):
    alpha = r_gw / (f_gw + f_sw)
    return alpha * (f_gw_new + f_sw_new)

def cod_concentration(pi, c_min=15, c_max=25, c_bar=20, pi_min=0.5, pi_max=1.5):
    if pi <= pi_min:
        return c_max
    elif pi >= pi_max:
        return c_min
    else:
        a = (c_max - c_min) / (pi_min - pi_max)
        b = c_bar - a
        return a * pi + b

def simulate(n_years, scenario, eta_gw, eta_sw, monte_carlo=False, n_sim=1000, file_input=None):
    params = SSP_PARAMS[scenario]
    gamma, shift_P, shift_E = params['gamma'], params['P_shift'], params['E_shift']
    
    if file_input and len(file_input.value) > 0:
        uploaded_file = list(file_input.value.values())[0]
        content = io.StringIO(uploaded_file['content'].decode('utf-8'))
        df_real = pd.read_csv(content)
        P, E, R, I = df_real['P'].values, df_real['E'].values, df_real['R'].values, df_real['I'].values
        n_years = len(P)
    else:
        P, E, R, I = generate_synthetic_data(n_years, 949.88, 131.31, shift_P, shift_E)
    
    P_bar, E_bar = 949.88, 492.86
    f_gw, f_sw, f_green = 0.5, 0.5, 0.2
    r_gw = 0.1

    if monte_carlo:
        ewei_samples = []
        for _ in range(n_sim):
            idx = np.random.randint(0, len(P))
            f_gw_new, f_sw_new, _ = adjust_agriculture_coeff(f_gw, f_sw, eta_gw, eta_sw, f_green, P[idx], E[idx], P_bar, E_bar, gamma)
            r_new = restitution_agriculture(f_gw_new, f_sw_new, f_gw, f_sw, r_gw)
            Rt_feas = feasible_runoff(R[idx], np.mean(R), MR=params['M'])
            It_feas = feasible_recharge(I[idx], np.mean(I))
            ek = (f_gw_new - r_new) + f_sw_new
            eweit = ek / (Rt_feas + It_feas) if Rt_feas + It_feas != 0 else np.nan
            ewei_samples.append(eweit)
        plt.hist(ewei_samples, bins=30, color='skyblue', edgecolor='black')
        plt.title(f"Monte Carlo Distribution of EWEI ({n_sim} runs)")
        plt.xlabel("EWEI")
        plt.ylabel("Frequency")
        plt.grid(True)
        plt.tight_layout()
        plt.show()
    else:
        results = []
        for i in range(n_years):
            f_gw_new, f_sw_new, _ = adjust_agriculture_coeff(f_gw, f_sw, eta_gw, eta_sw, f_green, P[i], E[i], P_bar, E_bar, gamma)
            r_new = restitution_agriculture(f_gw_new, f_sw_new, f_gw, f_sw, r_gw)
            pi = R[i] / np.mean(R)
            c_kt = cod_concentration(pi)
            Rt_feas = feasible_runoff(R[i], np.mean(R), MR=params['M'])
            It_feas = feasible_recharge(I[i], np.mean(I))
            ek = (f_gw_new - r_new) + f_sw_new
            eweit = ek / (Rt_feas + It_feas) if Rt_feas + It_feas != 0 else np.nan
            results.append((P[i], E[i], f_gw_new, f_sw_new, r_new, c_kt, eweit))
        df = pd.DataFrame(results, columns=["P", "E", "fk_gw", "fk_sw", "rk", "COD", "EWEI"])
        df.index.name = "Year"
        df.plot(subplots=True, layout=(3, 3), figsize=(12, 8), title="MUST4Water Hydrological Simulation")
        plt.tight_layout()
        plt.show()

file_input = FileUpload(accept='.csv', multiple=False)
interact(simulate,
         n_years=IntSlider(min=5, max=30, step=1, value=11, description="Years"),
         scenario=Dropdown(options=list(SSP_PARAMS.keys()), value='SSP2 (baseline)', description="Scenario"),
         eta_gw=FloatSlider(min=0.0, max=1.0, step=0.05, value=0.5, description="η_gw"),
         eta_sw=FloatSlider(min=0.0, max=1.0, step=0.05, value=0.5, description="η_sw"),
         monte_carlo=Checkbox(value=False, description="Run Monte Carlo"),
         n_sim=IntSlider(min=100, max=5000, step=100, value=1000, description="# Simulations"),
         file_input=io.StringIO())  # Replace with actual FileUpload when live
